In [ ]:
# Demonstration - Word List Creation

# A word list as a collection of moods and words associated with those moods - used to determine how a user is feeling based on the tokens
# in their notes. A user will be asked to enter a series of moods and then a series of words associated with those moods, which will then
# be stored locally in a .csv file. Keeping in line with the application's ethos of security, a word list must be stored locally as the
# contents are unencrypted.

moods = input("Moods: ").split()
words = []

for mood in moods:
    feelings = input(f"{mood}: ").split()
    words.append((feelings))

file = open("wl.csv", "w")

for idx, wordset in enumerate(words):
    values = ",".join(wordset) + "," + moods[idx]
    file.write(values + "\n")

file.close()

In [ ]:
# Demonstration - Word List Reading

# Once a user has created a word list, the contents of the file will need to be read and loaded into memory to be utilised.

moods = []
descriptors = []
moods_frequencies = {}

with open("wl.csv") as word_list:
    for entry in word_list:
        entry = entry.split(",")
        mood = entry[-1].strip("\n")
        words = entry[:-1]

        moods.append(mood)
        descriptors.append(words)

for mood in moods:
    moods_frequencies[mood] = 0

In [ ]:
# Demonstration - Note Analysis

# Now that the word list is loaded into memory, subnotes requested from the API can be analysed.

import string
import requests
import sys
import ast
from blowfish import Cipher

def get_cipher_key(token: str) -> bytes:
    data = {
        "token": token
    }

    key = requests.get(url + "/key", json=data).text
    return str.encode(key)


def get_iv(token: str) -> bytes:
    data = {
        "token": token
    }

    iv = requests.get(url + "/iv", json=data).text
    return str.encode(iv)

def decrypt_content(cipher: Cipher, encrypted_content: str, iv: str) -> str:
    encrypted_content = ast.literal_eval(encrypted_content)
    decrypted_content = b"".join(cipher.decrypt_cfb(encrypted_content, iv))

    return decrypted_content.decode()

def analyse(text):
    for word in text:
        for idx, descriptor in enumerate(descriptors):
            if word in descriptor:
                mood = moods[idx]
                moods_frequencies[mood] += 1
    
def determine_mood(moods_frequencies):
    sorted_moods_frequencies = sorted(moods_frequencies.items(), key=lambda x: x[1], reverse=True)
    mood = sorted_moods_frequencies[0][0]

    return mood

token = input("Token: ")
data = {"token": token}
url = "http://192.168.0.10:5000"
response = requests.post(url + "/validate-token", json=data)

if response.text == "0":
    print("Invalid token.")
    sys.exit()

subnote_id = int(input("Subnote ID: "))

data = {"token": token, "subnote_id": subnote_id}
response = requests.get(url + "/subnote", json=data)

if response.text == "Invalid subnote ID.":
    print(response.text)
    sys.exit()

if response.text == "You are not allowed to view this subnote.":
    print(response.text)
    sys.exit()

subnote = response.json()
content = subnote["content"]
key = get_cipher_key(token)
cipher = Cipher(key)
iv = get_iv(token)
content = decrypt_content(cipher, content, iv)
content = content.strip()
content = content.lower()
content = content.translate(str.maketrans('', '', string.punctuation))
content = content.split()

analyse(content)
print(moods_frequencies)
determined_mood = determine_mood(moods_frequencies).title()

print("Determined Mood: " + determined_mood)